## OpenSearch Hybrid 검색을 통한 RAG

#### 설정

In [42]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

### 1. Bedrock Client 생성

In [43]:
from utils import bedrock

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region='us-east-1'
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 2. Titan Embedding 및 LLM Claude-v2 모델 로딩

#### LLM 로딩

In [44]:
from langchain.llms import Bedrock
from langchain.callbacks.stdout import StdOutCallbackHandler

# Create Anthropic Model
llm_text = Bedrock(
  model_id = "anthropic:cluade-v2",
  client = boto3_bedrock,
  model_kwargs={
    "max_tokens_to_sample": 512
  },
  streaming=True,
  callbacks=[StdOutCallbackHandler()]
)

#### Embedding Model 선택

In [45]:
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
  model_id="amazon.titan-embed-text-v1",
  client=boto3_bedrock,
  region_name="us-east-1",
)

## 2. LangChain OpenSearch VectorStore 생성

### LangChain OpenSearch VectorStore 생성

In [46]:
# opensearch info
host = "localhost"
port = 9200
opensearch_endpoint = f"https://{host}:{port}"
http_auth = ("admin", "admin")
index_name = "genai-demo-index-v1"

ca_certs_path = 'root-ca.pem'
# Optional client certificates if you don't want to use HTTP basic authentication.
client_cert_path = 'admin.pem'
client_key_path = 'admin-key.pem'

### OpenSearch ReIndexig
- 기존 "index genai-demo-index-v1"을 "genai-demo-index-v1-with-tokenizer"로 
- nori tokenizer 추가

OpenSearch Client

In [47]:
from opensearchpy import OpenSearch

# Create the client with SSL/TLS enabled, but hostname verification disabled.
os_client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = http_auth,
    client_cert = client_cert_path,
    client_key = client_key_path,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

In [48]:
from pprint import pprint

index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                 'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                   'type': 'keyword'}},
                                                                                            'type': 'text'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                 'type': 'keyword'}},
                                                                                          't

In [49]:
new_index_name = f"{index_name}-with-tokenizer"
new_index_name

'genai-demo-index-v1-with-tokenizer'

In [50]:
# use nori tokenizer 사용
# analyzer_config = {
#   "tokenizer": "nori",
#   "tokenizer_type": "nori_tokenizer",
#   "char_filter": ["html_strip"],
#   "filter": ["nori_number", "nori_readingform", "lowercase"],
#   "decompound_mode": "mixed",
#   "discard_punctuation": "true"
# }

### index 수정: 형태소 분석기 사용 enablement

In [51]:
# nori 형태소 분석기를 custom analyzer 등록
index_info[index_name]["settings"]["analysis"] = {
  "tokenizer": {
    "nori": {
      "type": "nori_tokenizer",
      "decompound_mode": "mixed",
      "discard_punctuation": "true"
    }
  },
  "analyzer": {
    "my_analyzer": {
      "type": "custom",
      "char_filter": ["html_strip"],
      "tokenizer": "nori",
      "filter": ["nori_number", "nori_readingform", "lowercase"]  # token filter
    }
  }
}

# analyzer가 적용될 칼럼에 맞추어 수정
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]['mappings']['properties']['text']['search_analyzer'] = "my_analyzer"

# index 설정 변경 없음
index_info[index_name]["settings"]["index"] = {
  "number_of_shards": "5",
  "knn.algo_param": {"ef_search": "512"},
  "knn": True,
  "number_of_replicas": "2"
}

# del index alias
# del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]


In [52]:
pprint(new_index_info)

{'aliases': {},
 'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                         'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                           'type': 'keyword'}},
                                                                    'type': 'text'},
                                                         'timestamp': {'type': 'float'},
                                                         'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                         'type': 'keyword'}},
                                                                  'type': 'text'}}},
                             'text': {'analyzer': 'my_analyzer',
                                      'fields': {'keyword': {'ignore_above': 256,
                                                    

In [54]:
from utils.opensearch import opensearch_utils

index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

index_name=genai-demo-index-v1-with-tokenizer, exists=False
Index does not exist


In [ ]:
# create index
OpenSearch.

In [12]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
  opensearch_url=opensearch_endpoint,
  embedding_function=llm_emb,
  index_name =index_name,
  http_auth=http_auth,
  is_aoss=False,
  engine="faiss",
  space_type="l2",
  use_ssl=True,
  verify_certs=True,
  ssl_assert_hostname=False,
  ssl_show_warn=False,
  ca_certs=ca_certs_path
)


### OpenSearch Cleint 생성

In [ ]:
new_index_name